# Text-Guided Editing of Images (Using CLIP and StyleGAN)

In [ ]:
#@title Setup (may take a few minutes)
!git clone https://github.com/orpatashnik/StyleCLIP.git

import os
os.chdir(f'./StyleCLIP')

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT'
downloaded = drive.CreateFile({'id':file_id})
downloaded.FetchMetadata(fetch_all=True)
downloaded.GetContentFile(downloaded.metadata['title'])

Cloning into 'StyleCLIP'...


  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Cloning https://github.com/openai/CLIP.git to c:\users\marti\appdata\local\temp\pip-req-build-gri7y1xe
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.1/1.1 MB 4.3 MB/s eta 0:00:00
     -------------------------------------- 162.6/162.6 MB 3.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369447 sha256=83f6f14a1a648e94a80f70a966b7fe903fd5e8a16aeb2830664f493bb784c6d0
  Stored in directory: C:\Users\marti\AppData\Local\Temp\pip-ephem-wheel-cache-em4s1hsh\wheels\fd\b9\c3\5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\marti\AppData\Local\Temp\pip-req-build-gri7y1xe'


ModuleNotFoundError: No module named 'pydrive'

In [ ]:
experiment_type = 'edit' #@param ['edit', 'free_generation']

description = 'A person with purple hair' #@param {type:"string"}

latent_path = None #@param {type:"string"}

optimization_steps = 100 #@param {type:"number"}

l2_lambda = 0.008 #@param {type:"number"}

create_video = True #@param {type:"boolean"}

In [ ]:
#@title Additional Arguments
args = {
    "description": description,
    "ckpt": "stylegan2-ffhq-config-f.pt",
    "stylegan_size": 1024,
    "lr_rampup": 0.05,
    "lr": 0.1,
    "step": optimization_steps,
    "mode": experiment_type,
    "l2_lambda": l2_lambda,
    "latent_path": latent_path,
    "truncation": 0.7,
    "save_intermediate_image_every": 1 if create_video else 20,
    "results_dir": "results"
}

In [ ]:
from optimization.run_optimization import main
from argparse import Namespace
result = main(Namespace(**args))

In [ ]:
#@title Visualize Result
from torchvision.utils import make_grid
from torchvision.transforms import ToPILImage
result_image = ToPILImage()(make_grid(result.detach().cpu(), normalize=True, scale_each=True, range=(-1, 1), padding=0))
h, w = result_image.size
result_image.resize((h // 2, w // 2))

In [ ]:
#@title Create and Download Video

!ffmpeg -r 15 -i results/%05d.png -c:v libx264 -vf fps=25 -pix_fmt yuv420p out.mp4
from google.colab import files
files.download('out.mp4')